In [1]:
from _init_path import *

In [2]:
from src.load_data import load_data
import pandas as pd
import numpy as np

df2 = load_data()
df2.head()


Loading raw CSV file...


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [3]:
import numpy as np

# Make a working copy so we don't modify the original in memory
df = df2.copy()

# --- Clean / Convert ---
# Convert text date/time into actual pandas datetime64 objerts
df["publish_time"] = pd.to_datetime(df["publish_time"], errors="coerce")

# Forces these columns to be numeric, not strings
for col in ["views", "likes", "dislikes", "comment_count"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Remove rows that are missing critical numeric info
df = df.dropna(subset=["views", "likes", "comment_count"])

print("Rows after cleaning:", len(df))
df.head()

Rows after cleaning: 40949


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13 17:13:01+00:00,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13 07:30:00+00:00,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12 19:05:24+00:00,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13 11:00:04+00:00,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12 18:01:41+00:00,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [4]:
# Basic engagement metrics
df["like_ratio"] = df["likes"] / (df["views"] + 1)
df["comment_ratio"] = df["comment_count"] / (df["views"] + 1)

# Quantiles help you choose thresholds
df[["like_ratio", "comment_ratio"]].describe(percentiles=[0.25, 0.5, 0.75, 0.9])


,like_ratio,comment_ratio
count,40949.000000,40949.000000
mean,0.034413,0.004453
std,0.027008,0.005736
min,0.000000,0.000000
25%,0.014967,0.001607
50%,0.028273,0.002961
75%,0.046751,0.005214
90%,0.068272,0.009090
max,0.290466,0.117643


In [5]:
q75 = df["like_ratio"].quantile(0.75)
df["high_engagement"] = (df["like_ratio"] >= q75).astype(int)

df["high_engagement"].value_counts(normalize=True)


high_engagement
0    0.749982
1    0.250018
Name: proportion, dtype: float64

In [6]:
from pathlib import Path

processed_path = Path("../data/processed/youtube_engagement_clean.parquet")
processed_path.parent.mkdir(parents=True, exist_ok=True)

df.to_parquet(processed_path, index=False)
print("Saved cleaned dataset to", processed_path)


Saved cleaned dataset to ..\data\processed\youtube_engagement_clean.parquet
